In [1]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

<IPython.core.display.Javascript object>

In [3]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

<IPython.core.display.Javascript object>

In [4]:
SETUP = False

<IPython.core.display.Javascript object>

In [5]:
if SETUP:
    !pip install -q -U toai
    !pip install -q -U nb_black
    !pip install -q -U tensorflow-datasets
    !pip install -q -U --no-deps tensorflow-addons
    !pip install -q -U tensorflow_hub
    print(__import__("toai").__version__)
    print(__import__("tensorflow").__version__)

<IPython.core.display.Javascript object>

In [6]:
print(__import__("toai").__version__)

0.3.7


<IPython.core.display.Javascript object>

In [7]:
from toai.imports import *
from toai.utils import save_file, load_file
from toai.data import DataContainer, DataBundle
from toai.metrics import sparse_top_2_categorical_accuracy
from toai.models import save_keras_model, load_keras_model
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


<IPython.core.display.Javascript object>

In [8]:
DATA_DIR = Path("data/womens-ecommerce-clothing-reviews")
TEMP_DIR = Path("temp/womens-ecommerce-clothing-reviews")

<IPython.core.display.Javascript object>

In [9]:
if SETUP:
    shutil.rmtree(str(DATA_DIR), ignore_errors=True)
    shutil.rmtree(str(TEMP_DIR), ignore_errors=True)
    DATA_DIR.mkdir(parents=True)
    TEMP_DIR.mkdir(parents=True)
    kaggle.api.authenticate()
    kaggle.api.dataset_download_files(
        dataset="nicapotato/womens-ecommerce-clothing-reviews",
        path=DATA_DIR,
        unzip=True,
    )

<IPython.core.display.Javascript object>

In [10]:
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

<IPython.core.display.Javascript object>

In [11]:
all_data = pd.read_csv(
    DATA_DIR / "Womens Clothing E-Commerce Reviews.csv", low_memory=False
)

<IPython.core.display.Javascript object>

In [12]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23486 entries, 0 to 23485
Data columns (total 11 columns):
Unnamed: 0                 23486 non-null int64
Clothing ID                23486 non-null int64
Age                        23486 non-null int64
Title                      19676 non-null object
Review Text                22641 non-null object
Rating                     23486 non-null int64
Recommended IND            23486 non-null int64
Positive Feedback Count    23486 non-null int64
Division Name              23472 non-null object
Department Name            23472 non-null object
Class Name                 23472 non-null object
dtypes: int64(6), object(5)
memory usage: 2.0+ MB


<IPython.core.display.Javascript object>

In [13]:
all_data.describe(include="all")

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
count,23486.000000,23486.000000,23486.000000,19676,22641,23486.000000,23486.000000,23486.000000,23472,23472,23472
unique,NaN,NaN,NaN,13993,22634,NaN,NaN,NaN,3,6,20
top,NaN,NaN,NaN,Love it!,Perfect fit and i've gotten so many compliment...,NaN,NaN,NaN,General,Tops,Dresses
freq,NaN,NaN,NaN,136,3,NaN,NaN,NaN,13850,10468,6319
mean,11742.500000,918.118709,43.198544,NaN,NaN,4.196032,0.822362,2.535936,NaN,NaN,NaN
std,6779.968547,203.298980,12.279544,NaN,NaN,1.110031,0.382216,5.702202,NaN,NaN,NaN
min,0.000000,0.000000,18.000000,NaN,NaN,1.000000,0.000000,0.000000,NaN,NaN,NaN
25%,5871.250000,861.000000,34.000000,NaN,NaN,4.000000,1.000000,0.000000,NaN,NaN,NaN
50%,11742.500000,936.000000,41.000000,NaN,NaN,5.000000,1.000000,1.000000,NaN,NaN,NaN
75%,17613.750000,1078.000000,52.000000,NaN,NaN,5.000000,1.000000,3.000000,NaN,NaN,NaN


<IPython.core.display.Javascript object>

In [14]:
all_data.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


<IPython.core.display.Javascript object>

In [15]:
all_data["Rating"].value_counts()

5    13131
4     5077
3     2871
2     1565
1      842
Name: Rating, dtype: int64

<IPython.core.display.Javascript object>

In [16]:
def drop_values(df, col_name, values):
    return df.loc[~df[col_name].isin(values), :].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [17]:
def drop_rare_values(df, col_name, threshold):
    counts = df[col_name].value_counts(normalize=True)
    return df.loc[df[col_name].isin(counts[counts > threshold].index), :].reset_index(
        drop=True
    )

<IPython.core.display.Javascript object>

In [18]:
available_data = all_data[~all_data["Review Text"].isna()]

<IPython.core.display.Javascript object>

In [19]:
available_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22641 entries, 0 to 23485
Data columns (total 11 columns):
Unnamed: 0                 22641 non-null int64
Clothing ID                22641 non-null int64
Age                        22641 non-null int64
Title                      19675 non-null object
Review Text                22641 non-null object
Rating                     22641 non-null int64
Recommended IND            22641 non-null int64
Positive Feedback Count    22641 non-null int64
Division Name              22628 non-null object
Department Name            22628 non-null object
Class Name                 22628 non-null object
dtypes: int64(6), object(5)
memory usage: 2.1+ MB


<IPython.core.display.Javascript object>

In [20]:
available_data_bundle = DataBundle.from_dataframe(
    dataframe=available_data, x_col="Review Text", y_col="Rating"
)

<IPython.core.display.Javascript object>

In [21]:
train_data, valid_data, test_data = DataBundle.split(
    data_bundle=available_data_bundle, fracs=(0.8, 0.1, 0.1), random=False
)

<IPython.core.display.Javascript object>

In [22]:
label_map = train_data.make_label_map()

<IPython.core.display.Javascript object>

In [23]:
label_map

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

<IPython.core.display.Javascript object>

In [24]:
save_file(label_map, TEMP_DIR / "label_map.pickle")

<IPython.core.display.Javascript object>

In [25]:
label_map = load_file(TEMP_DIR / "label_map.pickle")

<IPython.core.display.Javascript object>

In [26]:
label_map

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4}

<IPython.core.display.Javascript object>

In [27]:
train_data.apply_label_map(label_map)

<IPython.core.display.Javascript object>

In [28]:
valid_data.apply_label_map(label_map)

<IPython.core.display.Javascript object>

In [29]:
test_data.apply_label_map(label_map)

<IPython.core.display.Javascript object>

In [30]:
train_data.value_counts()

{0: 653, 1: 1241, 2: 2278, 3: 3861, 4: 10080}

<IPython.core.display.Javascript object>

In [ ]:
# {0: 653, 1: 1241, 2: 2278, 3: 3861, 4: 10080}

In [31]:
class_weights = dict(
    enumerate(
        sk.utils.class_weight.compute_class_weight(
            "balanced", np.unique(train_data.y), train_data.y
        )
    )
)

<IPython.core.display.Javascript object>

In [32]:
class_weights

{0: 5.547626339969372,
 1: 2.9190975020145045,
 2: 1.5902546093064092,
 3: 0.9382543382543382,
 4: 0.3593849206349206}

<IPython.core.display.Javascript object>

In [33]:
train_data = DataBundle.from_unbalanced(train_data, 5000, train_data.value_counts())

<IPython.core.display.Javascript object>

In [34]:
train_data.value_counts()

{0: 5000, 1: 5000, 2: 5000, 3: 5000, 4: 5000}

<IPython.core.display.Javascript object>

In [35]:
@attr.s(auto_attribs=True)
class TextPreprocessor:
    max_length: int = 100
    default_value: str = b"<pad>"

    def __call__(self, text: tf.Tensor, label: tf.Tensor) -> tf.Tensor:
        text = tf.strings.regex_replace(text, b"[^a-zA-Z']", b" ")
        text = tf.strings.split(text)
        text = text[:, : self.max_length]
        return text.to_tensor(default_value=self.default_value), label

<IPython.core.display.Javascript object>

In [36]:
base_dataset = (
    train_data.to_dataset()
    .shuffle(len(train_data))
    .batch(BATCH_SIZE)
    .map(TextPreprocessor(), num_parallel_calls=AUTOTUNE)
)

<IPython.core.display.Javascript object>

In [37]:
def make_vocabulary(dataset):
    vocabulary = Counter()
    for x, _ in dataset:
        for review in x:
            vocabulary.update(review.numpy().tolist())
    return vocabulary

<IPython.core.display.Javascript object>

In [38]:
vocabulary = make_vocabulary(base_dataset)

<IPython.core.display.Javascript object>

In [39]:
len(vocabulary)

11311

<IPython.core.display.Javascript object>

In [40]:
vocabulary.most_common()[:10]

[(b'<pad>', 978354),
 (b'the', 89899),
 (b'i', 55026),
 (b'and', 51535),
 (b'it', 48626),
 (b'a', 45188),
 (b'is', 31933),
 (b'to', 27454),
 (b'this', 24139),
 (b'in', 22994)]

<IPython.core.display.Javascript object>

In [41]:
VOCABULARY_SIZE = 9000

<IPython.core.display.Javascript object>

In [42]:
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:VOCABULARY_SIZE]
]

<IPython.core.display.Javascript object>

In [43]:
len(truncated_vocabulary)

9000

<IPython.core.display.Javascript object>

In [44]:
word_to_id = {word: index for index, word in enumerate(truncated_vocabulary)}

<IPython.core.display.Javascript object>

In [45]:
for word in b"it was the most amazing dress".split():
    print(word_to_id.get(word) if word_to_id.get(word) is not None else VOCABULARY_SIZE)

4
11
1
282
490
17


<IPython.core.display.Javascript object>

In [46]:
words = tf.constant(truncated_vocabulary)

<IPython.core.display.Javascript object>

In [47]:
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)

<IPython.core.display.Javascript object>

In [48]:
n_oov_buckets = VOCABULARY_SIZE // 10

<IPython.core.display.Javascript object>

In [49]:
n_oov_buckets

900

<IPython.core.display.Javascript object>

In [50]:
table = tf.lookup.StaticVocabularyTable(
    tf.lookup.KeyValueTensorInitializer(words, word_ids), n_oov_buckets
)

<IPython.core.display.Javascript object>

In [51]:
table.lookup(tf.constant([b"These shoes are very patogus".split()]))

<tf.Tensor: id=101732, shape=(1, 5), dtype=int64, numpy=array([[ 369, 1235,   30,   24, 9781]])>

<IPython.core.display.Javascript object>

In [52]:
@attr.s(auto_attribs=True)
class WordEncoder:
    vocabulary_table: tf.lookup.StaticVocabularyTable

    def __call__(self, text: tf.Tensor, labels: tf.Tensor) -> tf.Tensor:
        return self.vocabulary_table.lookup(text), labels

<IPython.core.display.Javascript object>

In [53]:
train_dataset = (
    train_data.to_dataset()
    .shuffle(len(train_data))
    .batch(BATCH_SIZE)
    .map(TextPreprocessor(), num_parallel_calls=AUTOTUNE)
    .map(WordEncoder(vocabulary_table=table), num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .prefetch(AUTOTUNE)
)

<IPython.core.display.Javascript object>

In [54]:
valid_dataset = (
    valid_data.to_dataset()
    .batch(BATCH_SIZE)
    .map(TextPreprocessor(), num_parallel_calls=AUTOTUNE)
    .map(WordEncoder(vocabulary_table=table), num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(AUTOTUNE)
)

<IPython.core.display.Javascript object>

In [55]:
test_dataset = (
    test_data.to_dataset()
    .batch(BATCH_SIZE)
    .map(TextPreprocessor(), num_parallel_calls=AUTOTUNE)
    .map(WordEncoder(vocabulary_table=table), num_parallel_calls=AUTOTUNE)
    .cache()
    .prefetch(AUTOTUNE)
)

<IPython.core.display.Javascript object>

In [56]:
data_container = DataContainer(
    base=train_dataset,
    train=train_dataset,
    train_steps=math.ceil(len(train_data) / BATCH_SIZE),
    validation=valid_dataset,
    test=test_dataset,
    label_map=label_map,
)

<IPython.core.display.Javascript object>

In [57]:
for x, y in data_container.train.take(1):
    print(x.shape)
    print(y.shape)
    print(x[0])
    print(y[0])

(32, 100)
(32,)
tf.Tensor(
[  54  135   28   10    6   24  162 2162   46   39   30   15  124   19
  316    5  137   28 1120  288   49  209    1  128    6   87    3    1
   36    6  346 1187 1598    2  753   12  508   59   25   49  391   20
    1   89    6   15   31  106    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)


<IPython.core.display.Javascript object>

In [58]:
for x, y in data_container.validation.take(1):
    print(x.shape)
    print(y.shape)
    print(x[0])
    print(y[0])

(32, 100)
(32,)
tf.Tensor(
[3381 1612  361   10    1  714  327  285    3  565  446   84   12   27
  605  170   14    5  334 1179 4808   43   56  423  142    3    1  112
   55  114  161   12  436 1646  151  277   31   25   13  666    6  101
  157  893   42 1794  455  222    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0], shape=(100,), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)


<IPython.core.display.Javascript object>

In [59]:
def make_sequential_lstm_model(
    n_categories, embedding_size, lstm_size, lstm_dropout, dropout
):
    return keras.models.Sequential(
        [
            keras.layers.Embedding(
                VOCABULARY_SIZE + n_oov_buckets,
                embedding_size,
                mask_zero=True,
                input_shape=[None],
            ),
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    lstm_size, dropout=lstm_dropout, return_sequences=True
                )
            ),
            keras.layers.Bidirectional(
                keras.layers.LSTM(
                    lstm_size, dropout=lstm_dropout, return_sequences=False
                )
            ),
            keras.layers.Dropout(dropout),
            keras.layers.Dense(n_categories, activation=keras.activations.softmax),
        ]
    )

<IPython.core.display.Javascript object>

In [60]:
model = make_sequential_lstm_model(
    n_categories=data_container.n_classes,
    embedding_size=256,
    lstm_size=256,
    lstm_dropout=0.1,
    dropout=0.5,
)

<IPython.core.display.Javascript object>

In [61]:
model.compile(
    loss=keras.losses.sparse_categorical_crossentropy,
    optimizer=keras.optimizers.Adam(lr=1e-4),
    metrics=[
        keras.metrics.sparse_categorical_accuracy,
        sparse_top_2_categorical_accuracy,
    ],
)

<IPython.core.display.Javascript object>

In [62]:
history = model.fit(
    data_container.train,
    steps_per_epoch=data_container.train_steps // 5,
    validation_data=data_container.validation,
    epochs=20,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.3),
        keras.callbacks.EarlyStopping(patience=3, restore_best_weights=True),
    ],
)

Train for 156 steps, validate for 71 steps
Epoch 1/20
156/156 [==============================] - 27s 171ms/step - loss: 1.5600 - sparse_categorical_accuracy: 0.2837 - sparse_top_2_categorical_accuracy: 0.5194 - val_loss: 1.4153 - val_sparse_categorical_accuracy: 0.4561 - val_sparse_top_2_categorical_accuracy: 0.6698
Epoch 2/20
156/156 [==============================] - 10s 62ms/step - loss: 1.2874 - sparse_categorical_accuracy: 0.4002 - sparse_top_2_categorical_accuracy: 0.7061 - val_loss: 1.0986 - val_sparse_categorical_accuracy: 0.5347 - val_sparse_top_2_categorical_accuracy: 0.7881
Epoch 3/20
156/156 [==============================] - 10s 61ms/step - loss: 1.1765 - sparse_categorical_accuracy: 0.4551 - sparse_top_2_categorical_accuracy: 0.7833 - val_loss: 1.0495 - val_sparse_categorical_accuracy: 0.5483 - val_sparse_top_2_categorical_accuracy: 0.8106
Epoch 4/20
156/156 [==============================] - 10s 62ms/step - loss: 1.0912 - sparse_categorical_accuracy: 0.5166 - sparse_top_

<IPython.core.display.Javascript object>

In [63]:
model.evaluate(data_container.validation)

71/71 [==============================] - 1s 19ms/step - loss: 1.0369 - sparse_categorical_accuracy: 0.5572 - sparse_top_2_categorical_accuracy: 0.8278


[1.0368759900751248, 0.5571744, 0.8278146]

<IPython.core.display.Javascript object>

In [64]:
print(
    classification_report(
        np.concatenate([y.numpy() for _, y in data_container.validation]),
        model.predict(data_container.validation).argmax(axis=1),
    )
)

              precision    recall  f1-score   support

           0       0.34      0.33      0.33        89
           1       0.29      0.26      0.28       159
           2       0.32      0.45      0.38       280
           3       0.38      0.45      0.41       538
           4       0.81      0.68      0.74      1199

    accuracy                           0.56      2265
   macro avg       0.43      0.44      0.43      2265
weighted avg       0.60      0.56      0.57      2265



<IPython.core.display.Javascript object>

In [65]:
train_dataset = (
    train_data.to_dataset()
    .shuffle(len(train_data))
    .batch(BATCH_SIZE)
    .cache()
    .repeat()
    .prefetch(AUTOTUNE)
)

<IPython.core.display.Javascript object>

In [66]:
valid_dataset = valid_data.to_dataset().batch(BATCH_SIZE).cache().prefetch(AUTOTUNE)

<IPython.core.display.Javascript object>

In [67]:
test_dataset = test_data.to_dataset().batch(BATCH_SIZE).cache().prefetch(AUTOTUNE)

<IPython.core.display.Javascript object>

In [68]:
data_container = DataContainer(
    base=train_dataset,
    train=train_dataset,
    train_steps=math.ceil(len(train_data) / BATCH_SIZE),
    validation=valid_dataset,
    test=test_dataset,
    label_map=label_map,
)

<IPython.core.display.Javascript object>

In [69]:
def train_model(
    model,
    data_container,
    epochs,
    lrs=None,
    optimizers=None,
    patience=5,
    class_weights=None,
    verbose=1,
    log_dir=str(TEMP_DIR / "logs"),
):
    if optimizers is None:
        optimizers = [keras.optimizers.Adam(lr) for lr in lrs]
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizers[0],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        data_container.train,
        steps_per_epoch=data_container.train_steps,
        validation_data=data_container.validation,
        epochs=epochs[0],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=patience, factor=0.3),
            keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )
    model.layers[0].trainable = True
    model.compile(
        loss=keras.losses.sparse_categorical_crossentropy,
        optimizer=optimizers[1],
        metrics=[
            keras.metrics.sparse_categorical_accuracy,
            sparse_top_2_categorical_accuracy,
        ],
    )
    model.fit(
        data_container.train,
        steps_per_epoch=data_container.train_steps,
        validation_data=data_container.validation,
        epochs=epochs[1],
        callbacks=[
            keras.callbacks.ReduceLROnPlateau(patience=patience // 2, factor=0.3),
            keras.callbacks.EarlyStopping(patience=patience, restore_best_weights=True),
            keras.callbacks.TensorBoard(log_dir=log_dir),
        ],
        class_weight=class_weights,
        verbose=verbose,
    )

<IPython.core.display.Javascript object>

In [70]:
def make_hub_model(url, n_categories):
    return keras.Sequential(
        [
            hub.KerasLayer(url, dtype=tf.string, input_shape=[]),
            keras.layers.Dropout(0.5),
            keras.layers.Dense(n_categories, activation=keras.activations.softmax),
        ]
    )

<IPython.core.display.Javascript object>

In [77]:
def run_models(urls, data_container, class_weights):
    for url in urls:
        model = make_hub_model(url, data_container.n_classes)
        model_name = f"{url.split('/')[4]}"
        print(f" {model_name} ".center(80, "="))
        shutil.rmtree(str(TEMP_DIR / model_name), ignore_errors=True)
        train_model(
            model=model,
            data_container=data_container,
            epochs=[25, 15],
            optimizers=[keras.optimizers.Adam(lr=3e-4), keras.optimizers.Adam(lr=1e-4)],
            class_weights=class_weights,
            patience=4,
            verbose=2,
            log_dir=str(TEMP_DIR / model_name),
        )
        model.save(f"{TEMP_DIR / model_name}.h5")
        save_keras_model(
            model,
            str(TEMP_DIR / model_name / "architecture"),
            str(TEMP_DIR / model_name / "weights"),
        )
        keras.backend.clear_session()
        del model
        keras.backend.clear_session()

<IPython.core.display.Javascript object>

In [78]:
def evaluate_models(urls, data_container):
    reports = {}
    for url in urls:
        model_name = f"{url.split('/')[4]}"
        print(f" {model_name} ".center(80, "="))
        try:
            model = keras.model.load_model(
                f"{TEMP_DIR / model_name}.h5",
                custom_objects={"KerasLayer": hub.KerasLayer},
            )
        except:
            print(f"Loading architecture & weights separately")
            model = load_keras_model(
                str(TEMP_DIR / model_name / "architecture"),
                str(TEMP_DIR / model_name / "weights"),
                custom_objects={"KerasLayer": hub.KerasLayer},
            )
        reports[model_name] = classification_report(
            [
                label.numpy()
                for _, label in data_container.validation.take(-1).unbatch()
            ],
            model.predict(data_container.validation).argmax(axis=1),
        )
        del model
    return reports

<IPython.core.display.Javascript object>

In [79]:
model_urls = (
    #     "https://tfhub.dev/google/Wiki-words-250/2",
    "https://tfhub.dev/google/Wiki-words-250-with-normalization/2",
    "https://tfhub.dev/google/nnlm-en-dim128-with-normalization/2",
)

<IPython.core.display.Javascript object>

In [ ]:
run_models(model_urls, data_container, class_weights)

====================== Wiki-words-250-with-normalization =======================
Train for 782 steps, validate for 71 steps
Epoch 1/25
782/782 - 4s - loss: 1.6490 - sparse_categorical_accuracy: 0.2159 - sparse_top_2_categorical_accuracy: 0.4261 - val_loss: 1.5699 - val_sparse_categorical_accuracy: 0.3210 - val_sparse_top_2_categorical_accuracy: 0.5638
Epoch 2/25
782/782 - 4s - loss: 1.5953 - sparse_categorical_accuracy: 0.2564 - sparse_top_2_categorical_accuracy: 0.4773 - val_loss: 1.5226 - val_sparse_categorical_accuracy: 0.4283 - val_sparse_top_2_categorical_accuracy: 0.6402
Epoch 3/25
782/782 - 4s - loss: 1.5689 - sparse_categorical_accuracy: 0.2773 - sparse_top_2_categorical_accuracy: 0.5083 - val_loss: 1.5040 - val_sparse_categorical_accuracy: 0.4384 - val_sparse_top_2_categorical_accuracy: 0.6508
Epoch 4/25
782/782 - 4s - loss: 1.5420 - sparse_categorical_accuracy: 0.2989 - sparse_top_2_categorical_accuracy: 0.5378 - val_loss: 1.4828 - val_sparse_categorical_accuracy: 0.4486 - va

In [75]:
reports = evaluate_models(model_urls, data_container)

====================== Wiki-words-250-with-normalization =======================
Loading architecture & weights separately
====================== nnlm-en-dim128-with-normalization =======================
Loading architecture & weights separately


<IPython.core.display.Javascript object>

In [76]:
for model_name, report in reports.items():
    print(f" {model_name} ".center(80, "="))
    print(report)

====================== Wiki-words-250-with-normalization =======================
              precision    recall  f1-score   support

           0       0.31      0.37      0.34        89
           1       0.25      0.31      0.28       159
           2       0.32      0.37      0.34       280
           3       0.45      0.51      0.48       538
           4       0.86      0.73      0.79      1199

    accuracy                           0.59      2265
   macro avg       0.44      0.46      0.44      2265
weighted avg       0.63      0.59      0.60      2265

====================== nnlm-en-dim128-with-normalization =======================
              precision    recall  f1-score   support

           0       0.30      0.38      0.34        89
           1       0.26      0.33      0.29       159
           2       0.34      0.38      0.35       280
           3       0.45      0.52      0.48       538
           4       0.85      0.73      0.79      1199

    accuracy           

<IPython.core.display.Javascript object>

In [ ]:
# ================================ Wiki-words-250 ================================
#               precision    recall  f1-score   support

#            0       0.54      0.81      0.65        77
#            1       0.52      0.68      0.59       159
#            2       0.54      0.60      0.57       303
#            3       0.48      0.54      0.51       500
#            4       0.86      0.73      0.79      1226

#     accuracy                           0.67      2265
#    macro avg       0.59      0.67      0.62      2265
# weighted avg       0.70      0.67      0.68      2265

# ====================== Wiki-words-250-with-normalization =======================
#               precision    recall  f1-score   support

#            0       0.63      0.83      0.72        77
#            1       0.55      0.69      0.61       159
#            2       0.53      0.61      0.57       303
#            3       0.50      0.57      0.53       500
#            4       0.87      0.75      0.81      1226

#     accuracy                           0.69      2265
#    macro avg       0.62      0.69      0.65      2265
# weighted avg       0.71      0.69      0.70      2265

# ====================== nnlm-en-dim128-with-normalization =======================
#               precision    recall  f1-score   support

#            0       0.61      0.86      0.71        77
#            1       0.56      0.69      0.62       159
#            2       0.54      0.59      0.56       303
#            3       0.49      0.56      0.52       500
#            4       0.87      0.74      0.80      1226

#     accuracy                           0.68      2265
#    macro avg       0.61      0.69      0.64      2265
# weighted avg       0.71      0.68      0.69      2265

In [ ]:
# ================================ Wiki-words-250 ================================
#               precision    recall  f1-score   support

#            0       0.54      0.81      0.65        77
#            1       0.52      0.68      0.59       159
#            2       0.54      0.60      0.57       303
#            3       0.48      0.54      0.51       500
#            4       0.86      0.73      0.79      1226

#     accuracy                           0.67      2265
#    macro avg       0.59      0.67      0.62      2265
# weighted avg       0.70      0.67      0.68      2265

# ====================== Wiki-words-250-with-normalization =======================
#               precision    recall  f1-score   support

#            0       0.63      0.83      0.72        77
#            1       0.55      0.69      0.61       159
#            2       0.53      0.61      0.57       303
#            3       0.50      0.57      0.53       500
#            4       0.87      0.75      0.81      1226

#     accuracy                           0.69      2265
#    macro avg       0.62      0.69      0.65      2265
# weighted avg       0.71      0.69      0.70      2265

# ====================== nnlm-en-dim128-with-normalization =======================
#               precision    recall  f1-score   support

#            0       0.61      0.86      0.71        77
#            1       0.56      0.69      0.62       159
#            2       0.54      0.59      0.56       303
#            3       0.49      0.56      0.52       500
#            4       0.87      0.74      0.80      1226

#     accuracy                           0.68      2265
#    macro avg       0.61      0.69      0.64      2265
# weighted avg       0.71      0.68      0.69      2265